In [1]:
# Download Libraries
%pip install google-cloud-storage
%pip install pyarrow # Apache Arrow
%pip install boto3
%pip install pandas psycopg2 
%pip install --upgrade psycopg2
%pip install sqlalchemy-redshift
%pip install google-cloud-bigquery


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#'
You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


...

You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Baruch_Student\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
# Import Libraries
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import io
from io import StringIO
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage
import os
import datetime
#from google.cloud import bigquery
import unicodedata
from sqlalchemy import create_engine
from google.cloud import bigquery
from google.oauth2 import service_account

In [6]:
#Gathering Data
#Extracting from datasource to view the head 

URL = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Define the data type for column 17 because it contains mixed data types including NaN
dtype_mapping = {'Exam No': 'object', 'List No': 'object', 'List Title Code': 'object', 'Group No': 'object', 'List Agency Code': 'object', 'List Div Code': 'object', 'Veteran Credit': 'object'}


# Specify NaN values to be considered as missing values
na_values = ['NaN', '', 'NA', 'nan']

df_raw = pd.read_csv(URL, dtype=dtype_mapping, na_values=na_values, low_memory=False)

print(df_raw.info())
print('\n')
print(df_raw.shape)
print('\n')
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491645 entries, 0 to 491644
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Exam No             491645 non-null  object 
 1   List No             491645 non-null  object 
 2   First Name          491638 non-null  object 
 3   MI                  272377 non-null  object 
 4   Last Name           491637 non-null  object 
 5   Adj. FA             491645 non-null  float64
 6   List Title Code     491645 non-null  object 
 7   List Title Desc     491645 non-null  object 
 8   Group No            491645 non-null  object 
 9   List Agency Code    491645 non-null  object 
 10  List Agency Desc    491645 non-null  object 
 11  List Div Code       0 non-null       object 
 12  Published Date      221816 non-null  object 
 13  Established Date    476876 non-null  object 
 14  Anniversary Date    476876 non-null  object 
 15  Extension Date      222939 non-nul

,Exam No,List No,First Name,MI,Last Name,Adj. FA,List Title Code,List Title Desc,Group No,List Agency Code,List Agency Desc,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit
0,7001,17897.000,MACDANIEL,NaN,CHARLES,94.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
1,3055,52.000,MARIUSZ,NaN,PRZEZDZIECKI,85.85,92005,CARPENTER,000,000,OPEN COMPETITIVE,NaN,04/10/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1111,634.000,TERRENCE,NaN,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),000,000,OPEN COMPETITIVE,NaN,05/18/2022,04/19/2023,04/19/2027,NaN,NaN,NaN,NaN,NaN
3,7001,21755.000,DANIEL,NaN,MELENDEZ,93.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,Residency Credit
4,7001,29676.000,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,000,000,OPEN COMPETITIVE,NaN,06/13/2018,02/27/2019,02/27/2023,02/27/2025,NaN,NaN,NaN,NaN


In [22]:
#Storing Data
# Specify the path to your service account key file
#service_account_key_file = '/Users/karmayangchentenzin/Downloads/Service_Key_HW.json'
service_account_key_file = 'service_acc_key.json' 

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Fetch data from the web (CSV file)
url = 'https://data.cityofnewyork.us/api/views/vx8i-nprf/rows.csv?accessType=DOWNLOAD'

# Read the contents of the CSV file
response = requests.get(url)

# Check if the request was successful
if response.status_code != 200:
    print("Failed to fetch CSV file.")
    exit()

# Read the CSV data into a DataFrame
try: 
    df = pd.read_csv(io.StringIO(response.text), dtype=dtype_mapping, na_values=na_values, low_memory=False)

    # Add timestamp column
    df['load_date'] = datetime.datetime.now() #Date the data is stored. A new column named "load_data" is created.

    # Replace periods with underscores in column names #This had to be done before the original data was loaded from the source to GC because when creating tables in BigQuery, an error message was shown saying that field names can not contain a period (.)
    df.columns = df.columns.str.replace('.', '_')

    # Convert DataFrame to PyArrow Table
    table = pa.Table.from_pandas(df)

    # Write PyArrow Table to Parquet format
    parquet_file_name = 'ny_civil_service_exam.parquet'  # name for the Parquet file
    pq.write_table(table, parquet_file_name)

    # Upload Parquet file to Google Cloud Storage
    bucket_name = 'cis4400_hw1_kyt'
    blob_name = parquet_file_name

    client = storage.Client.from_service_account_json(service_account_key_file)
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(parquet_file_name)

    print(f"Parquet file '{parquet_file_name}' uploaded to {bucket_name}/{blob_name} in Google Cloud Storage.")
except Exception as e:
    print(f"Error occurred: {e}")

Parquet file 'ny_civil_service_exam.parquet' uploaded to cis4400_hw1_kyt/ny_civil_service_exam.parquet in Google Cloud Storage.


In [4]:
# Extracting the data from Google Cloud

service_account_key_file = 'service_acc_key.json' 

# Optionally, load other configuration settings from the JSON file
with open(service_account_key_file, 'r') as f:
    config = json.load(f)

# Initialize Google Cloud Storage client
client = storage.Client.from_service_account_json(service_account_key_file)

# Specify the bucket name and Parquet file name
bucket_name = 'cis4400_hw1_kyt'
parquet_file_name = 'ny_civil_service_exam.parquet'

# Download Parquet file from Google Cloud Storage
bucket = client.bucket(bucket_name)
blob = bucket.blob(parquet_file_name)
blob.download_to_filename(parquet_file_name)

# Read Parquet file into a PyArrow Table
table = pq.read_table(parquet_file_name)

# Convert PyArrow Table back to DataFrame
parquet_df = table.to_pandas()

# Print the shape of the DataFrame
print("Shape of DataFrame:", parquet_df.shape)

# Make a copy of the DataFrame
df_copy = parquet_df.copy()

Shape of DataFrame: (491645, 21)


In [46]:
# Just for testing purpose to check if the data in the original match with the datawarehouse later
# Filter DataFrame to only include rows where first name is 'Jonathan' and last name is 'Thompson'
filtered_rows = df_copy[(df_copy['First Name'] == 'JONATHAN') & (df_copy['Last Name'] == 'THOMPSON') & (df_copy['MI'] == 'X')]

# Print the filtered rows
print("Results for first name '...' and last name '...':")
print(filtered_rows)


Results for first name 'Amazu' and last name 'Eqlo':
       Exam No    List No First Name MI Last Name  Adj_ FA List Title Code  \
111121    7001  20802.000   JONATHAN  X  THOMPSON     93.0           70310   

       List Title Desc Group No List Agency Code  ... List Div Code  \
111121     FIREFIGHTER      000              000  ...          None   

       Published Date Established Date Anniversary Date Extension Date  \
111121     06/13/2018       02/27/2019       02/27/2023     02/27/2025   

       Veteran Credit Parent Lgy Credit Sibling Lgy Credit  Residency Credit  \
111121           None              None               None  Residency Credit   

                        load_date  
111121 2024-05-02 06:50:53.299549  

[1 rows x 21 columns]


In [39]:
df_copy.columns

Index(['Exam No', 'List No', 'First Name', 'MI', 'Last Name', 'Adj_ FA',
       'List Title Code', 'List Title Desc', 'Group No', 'List Agency Code',
       'List Agency Desc', 'List Div Code', 'Published Date',
       'Established Date', 'Anniversary Date', 'Extension Date',
       'Veteran Credit', 'Parent Lgy Credit', 'Sibling Lgy Credit',
       'Residency Credit', 'load_date'],
      dtype='object')

In [19]:
df_copy.head()

,Exam No,List No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,Group No,List Agency Code,...,List Div Code,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,7001,17897.000,MACDANIEL,None,CHARLES,94.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-05-02 06:50:53.299549
1,3055,52.000,MARIUSZ,None,PRZEZDZIECKI,85.85,92005,CARPENTER,000,000,...,None,04/10/2024,None,None,None,None,None,None,None,2024-05-02 06:50:53.299549
2,1111,634.000,TERRENCE,None,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),000,000,...,None,05/18/2022,04/19/2023,04/19/2027,None,None,None,None,None,2024-05-02 06:50:53.299549
3,7001,21755.000,DANIEL,None,MELENDEZ,93.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,Residency Credit,2024-05-02 06:50:53.299549
4,7001,29676.000,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,000,000,...,None,06/13/2018,02/27/2019,02/27/2023,02/27/2025,None,None,None,None,2024-05-02 06:50:53.299549


In [5]:
#Tranformation 

# transformation code 1
df_copy['List No'] = df_copy['List No'].astype(str).str.replace(r'\..*', '', regex=True).astype(int)

# transformation code 2
# List of date columns to be formatted
date_columns = ['Published Date', 'Established Date', 'Anniversary Date', 'Extension Date']

# Convert date columns to datetime objects and format them
for col in date_columns:
    df_copy[col] = pd.to_datetime(df_copy[col]).dt.strftime('%Y-%m-%d')

#transformation code 3
date_column = 'load_date'  

# Convert the date column to datetime object and format it
df_copy[date_column] = pd.to_datetime(df_copy[date_column]).dt.strftime('%Y-%m-%d %H:%M:%S')

# transformation code 4
df_copy = df_copy.drop(columns=['List No', 'Group No', 'List Agency Code', 'List Div Code'])

#transformation code 5
# Drop rows with null values in the 'Adj_FA' field
df_copy = df_copy.dropna(subset=['Adj_ FA'])

# transformation code 6
# Remove duplicate rows from the DataFrame
df_copy = df_copy.drop_duplicates()

df_copy.head()

,Exam No,First Name,MI,Last Name,Adj_ FA,List Title Code,List Title Desc,List Agency Desc,Published Date,Established Date,Anniversary Date,Extension Date,Veteran Credit,Parent Lgy Credit,Sibling Lgy Credit,Residency Credit,load_date
0,7001,MACDANIEL,None,CHARLES,94.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,Residency Credit,2024-05-02 06:50:53
1,3055,MARIUSZ,None,PRZEZDZIECKI,85.85,92005,CARPENTER,OPEN COMPETITIVE,2024-04-10,NaN,NaN,NaN,None,None,None,None,2024-05-02 06:50:53
2,1111,TERRENCE,None,HAYES,70.00,13652,CERTIFIED IT ADMINISTRATOR (LAN/WAN),OPEN COMPETITIVE,2022-05-18,2023-04-19,2027-04-19,NaN,None,None,None,None,2024-05-02 06:50:53
3,7001,DANIEL,None,MELENDEZ,93.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,Residency Credit,2024-05-02 06:50:53
4,7001,KAREEM,P,MYERS,89.00,70310,FIREFIGHTER,OPEN COMPETITIVE,2018-06-13,2019-02-27,2023-02-27,2025-02-27,None,None,None,None,2024-05-02 06:50:53


In [37]:
print(df_copy.shape) #This shows that there are some rows that are duplicates because the number or rows differs from the original.

(491536, 17)


In [10]:
#  Getting the dataframe as csv file
df_copy.to_csv('cleaned_data.csv', index=False)

In [12]:
#Loading data into BigQuery data warehouse

# Set up your Google Cloud project ID
project_id = 'avian-silicon-418821'

# Set the path to your service account key file
service_account_key_file = 'service_acc_key.json'

# Set up BigQuery client with service account credentials
client = bigquery.Client.from_service_account_json(service_account_key_file, project=project_id)

# Define your dataset ID
dataset_id = 'cis4400_hw1'

# Create the BigQuery dataset if it doesn't exist
dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset = client.create_dataset(dataset, exists_ok=True)  # Use exists_ok=True to avoid errors if the dataset already exists

# Set up BigQuery table information
table_id = 'NY_CivilService_ExamScores'

# Create the BigQuery table with schema auto-detection (without specifying schema)
table_ref = dataset_ref.table(table_id)
job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Automatically detect schema from CSV file
    skip_leading_rows=1,  # Skip the header row
    source_format=bigquery.SourceFormat.CSV
)

# Load CSV file into BigQuery table
with open('cleaned_data.csv', 'rb') as source_file:
    job = client.load_table_from_file(
        source_file,
        table_ref,
        job_config=job_config
    )

job.result()  # Waits for the job to complete

print(f"Loaded {job.output_rows} rows into {dataset_id}.{table_id}.")


Loaded 491536 rows into cis4400_hw1.NY_CivilService_ExamScores.


In [44]:
# Testing Purposes only
# Filter DataFrame to only include rows where first name is 'Amazu' and last name is 'Eqlo'
filtered_rows = df_copy[(df_copy['First Name'] == 'JONATHAN') & (df_copy['Last Name'] == 'THOMPSON') & (df_copy['MI'] == 'X')]

# Print the filtered rows
print("Results for first name 'Amazu' and last name 'Eqlo':")
print(filtered_rows)


Results for first name 'Amazu' and last name 'Eqlo':
       Exam No First Name MI Last Name  Adj_ FA List Title Code  \
111121    7001   JONATHAN  X  THOMPSON     93.0           70310   

       List Title Desc  List Agency Desc Published Date Established Date  \
111121     FIREFIGHTER  OPEN COMPETITIVE     2018-06-13       2019-02-27   

       Anniversary Date Extension Date Veteran Credit Parent Lgy Credit  \
111121       2023-02-27     2025-02-27           None              None   

       Sibling Lgy Credit  Residency Credit            load_date  
111121               None  Residency Credit  2024-05-02 06:50:53  


In [36]:
# Some more transformation and loading of data into each table in BigQuery
# Create a BigQuery client
client = bigquery.Client()

# Define the BigQuery dataset and table name
dataset_id = 'cis4400_hw1'
table_name = 'dim_candidate_exam'

# Rename columns in the DataFrame to match the column names in BigQuery table
df_copy.rename(columns={'First Name': 'first_name', 'MI': 'middle_initial', 'Last Name': 'last_name', 'Exam No': 'exam_id'}, inplace=True)

# Filter DataFrame to only include specified columns
df_to_load = df_copy[['exam_id', 'first_name', 'middle_initial', 'last_name']]

# Define the table reference
table_ref = client.dataset(dataset_id).table(table_name)

# Load data into the BigQuery table
job = client.load_table_from_dataframe(df_to_load, table_ref)

# Wait for the job to complete
job.result()

print(f"Data loaded successfully into BigQuery table '{dataset_id}.{table_name}'.")


Data loaded successfully into BigQuery table 'cis4400_hw1.dim_candidate_exam'.


In [57]:
# Lookup tabel for dim_list_title dimension table
# Filter DataFrame to only include specified columns
df_to_load = df_copy[['list_title_id', 'list_title_description']]

# Drop duplicate rows based on 'list_title_id'
df_to_load_unique = df_to_load.drop_duplicates(subset=['list_title_id'])

# Define the table reference
table_ref = client.dataset(dataset_id).table(table_name)

# Load data into the BigQuery table
job = client.load_table_from_dataframe(df_to_load_unique, table_ref)

# Wait for the job to complete
job.result()

# Print the unique rows as a lookup table
print("Lookup table for 'list_title_id' and 'list_title_description':")
df_to_load_unique.head()
print(df_to_load_unique.shape)


Lookup table for 'list_title_id' and 'list_title_description':
(530, 2)


In [58]:
# Load the lookup into the table in BigQuery
# Define the BigQuery dataset and table name for the lookup table
table_name = 'dim_list_title'

# Define the table reference for the lookup table
lookup_table_ref = client.dataset(dataset_id).table(table_name)

# Load data into the BigQuery table for the lookup table
lookup_job = client.load_table_from_dataframe(df_to_load_unique, lookup_table_ref)

# Wait for the job to complete
lookup_job.result()

print(f"Lookup table loaded successfully into BigQuery table '{dataset_id}.{table_name}'.")


Lookup table loaded successfully into BigQuery table 'cis4400_hw1.dim_list_title'.
